In [ ]:
%matplotlib inline
import flopy
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl
import pickle
import random
from scipy.fft import fft, fftfreq
import sys
import xarray
import xugrid

In [ ]:
sys.path.append("../common")
from liss_settings import cx, cx_provider, extent, boxx, boxy, extentmax, fig_ext, transparent

In [ ]:
destination_crs = "EPSG:32618"

In [ ]:
ws = pl.Path("figures-observations")
ws.mkdir(exist_ok=True, parents=True)

### Stations to be plotted in the summary comparison plot

In [ ]:
pstations = (
    "Battery",
    "01311875",
    "Eaton",
    "01309225",
    "Silver Eel Pond",
    "Montauk",
)
ostations = (
    "Long Island Sound",
    "Great Peconic Bay",
    "New York Bight",
)

### Read the observation locations and create an observation dictionary

In [ ]:
sfincs_path = "../sfincs/test_north_fork/base/sfincs.obs"

In [ ]:
obs_dict = {}
with open(sfincs_path, "r") as f:
    lines = f.readlines()
for line in lines[:]:
    t = line.rstrip().split()
    key = " ".join(t[2:]).replace('"', '')
    if key in ("great_peconic_bay", "long_island_sound", "new_york_bight"):
        key = key.replace("_", " ").title()
    elif key.startswith("usgs_"):
        key = key.replace("usgs_", "")
    obs_dict[key] = (float(t[0]), float(t[1]))
obs_dict

In [ ]:
x, y, v, pc = [], [], [], []
xytext_locs = {}
for key, (xx, yy) in obs_dict.items():
    x.append(xx)
    y.append(yy)
    v.append(key)
    if key in pstations:
        pc.append("green")
    elif key in ostations:
        pc.append("blue")
    else:
        pc.append("black")
    dx, dy = 3e3, -1.2e4
    if "eaton" in key.lower():
        dy *= -1.0
    elif key in ("New London",):
        dx *= 7.0
    elif key in ("Battery", "New Rochelle"):
        dx *= -1.0
        dy *= -1.0
    elif key in ("01302845",):
        dx *= 0.65
        dy *= -0.65
    elif key in ("Willets Point",):
        dx *= -6.0
        dy *= -0.75
    elif key in ("Kings Point",):
        dx *= -5.0
        dy *= -1.0
    elif key in ("01311850", ):
        dx *= -0.5
        dy *= -0.5
    elif key in ("01311875", "South Jamesport"):
        dx *= 2.0
        dy *= 2.0
    elif key in ("01311145",):
        dx *= 1.5
        dy *= 1.5
    elif key in ("01310740",):
        dx *= 4.0
        dy *= 1.15
    elif key in ("01310521",):
        dx *= 0.65
        dy *= -0.65
    elif "jefferson" in key.lower():
        dy *= -0.7
    elif key in ("Great Peconic Bay",):
        dy *= -1.0
    elif key in ("Long Island Sound",):
        dy *= 1.15
    elif key in ("New York Bight",):
        dx *= 5.0
        
    xytext_locs[key] = (xx + dx , yy + dy)
x = np.array(x)
y = np.array(y)

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, ax = plt.subplots(
        layout="constrained",
        figsize=(6.85, 4.9),
        )
    ax.set_xlim(extentmax[:2])
    ax.set_ylim(extentmax[2:])    
    ax.scatter(x, y, c=pc)
    for idx, (xx, yy, s) in enumerate(zip(x,y,v)):
        flopy.plot.styles.add_annotation(
            xy=(xx, yy), 
            xytext=xytext_locs[s], 
            xycoords="data", 
            text=s, 
            bold=False, 
            fontsize=6,
            ha="center",
            va="center",
            arrowprops=dict(
                arrowstyle= "-",
                shrinkA=1,
                shrinkB=1,
            )
        )
    cx.add_basemap(ax, crs=destination_crs, attribution=False, source=cx_provider)

    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticklabels([])
    ax.set_yticklabels([])    
    
    fig.savefig(ws / f"obs_locations{fig_ext}", dpi=300, transparent=transparent)

### Load the history files

D-FLOW FM file

In [ ]:
source_path = "../dflow-fm/coarse/tides/run/output/FlowFM_his.nc"
dflow_ds = xugrid.open_dataset(source_path)
dflow_ds

SFINCS

In [ ]:
source_path = "../sfincs/test_north_fork/run/sfincs_his.nc"
sfincs_ds = xugrid.open_dataset(source_path)
sfincs_ds

### Create a pandas observation for each observation location

In [ ]:
dflow_drop = ["station_x_coordinate", "station_y_coordinate", "station_name"]
dflow_rename = {"waterlevel": "D-FLOW FM"}

In [ ]:
sfincs_drop = ["station_id", "station_name", "point_x", "point_y"]
sfincs_rename = {"point_zs": "SFINCS"}

In [ ]:
sfincs_obs = {}
dflow_obs = {}
for idx, key in enumerate(obs_dict.keys()):
    subset = sfincs_ds["point_zs"].loc[:, idx]
    print(f"{idx+1:02d}-1: {key} ({subset.size})")
    if subset.values.size > 0:
        df = subset.to_dataframe()
        df.rename(columns=sfincs_rename, inplace=True)
        df.drop(labels=sfincs_drop, inplace=True, axis=1)
        sfincs_obs[key] = df.copy()
    else:
        sfincs_obs[key] = None
    subset = dflow_ds["waterlevel"].loc[:, idx]
    print(f"{idx+1:02d}-2: {key} ({subset.size})")
    if subset.values.size > 0:
        df = subset.to_dataframe()
        df.rename(columns=dflow_rename, inplace=True)
        df.drop(labels=dflow_drop, inplace=True, axis=1)
        dflow_obs[key] = df.copy()
    else:
        dflow_obs[key] = None

### Plot each observation location

In [ ]:
line_weight = 0.5

In [ ]:
# key = "usgs_01311875"
for idx, key in enumerate(obs_dict.keys()):
    print(f"{idx+1:02d}: {key}")
    with flopy.plot.styles.USGSPlot():
        fig, axs = plt.subplots(
            ncols=1,
            nrows=2,
            layout="constrained",
            figsize=(6, 4),
            sharex=True,
            )

        s_df = sfincs_obs[key]
        d_df = dflow_obs[key]
        if s_df is None and d_df is None:
            continue

        ax = axs[0]
        if s_df is not None:
            sfincs_obs[key].plot(ax=ax, lw=line_weight, color="blue", legend=False)
        if d_df is not None:
            dflow_obs[key].plot(ax=ax, lw=line_weight, color="red", legend=False)
        ax.set_ylabel("Stage, m")
        flopy.plot.styles.heading(ax=ax, heading=f"Station: {key}")
        flopy.plot.styles.graph_legend(ax=ax, loc="lower right")
        
        if d_df is not None:
            df = d_df.copy()
        else:
            df = s_df.copy()
        if s_df is not None and d_df is not None:
            df["difference"] = dflow_obs[key]["D-FLOW FM"].values - sfincs_obs[key]["SFINCS"].values
        else:
            df["difference"] = np.nan

        ax = axs[1]
        df["difference"].plot(ax=ax, lw=line_weight, color="black")
        ax.set_ylabel("Stage Difference, m")
    
        fig.savefig(ws / f"obs_{key}{fig_ext}", dpi=300, transparent=transparent)
    plt.close()


### Plot select observation locations (ostations)

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(
        ncols=3,
        nrows=2,
        layout="constrained",
        figsize=(11.5, 2),
        sharex=True,
        sharey=True
        )

    idx = 0
    jdx = 0

    for key in ostations:

        s_df = sfincs_obs[key]
        d_df = dflow_obs[key]
    
        ax = axs[idx, jdx]
        if s_df is not None:
            sfincs_obs[key].plot(ax=ax, lw=line_weight, color="blue", legend=False, xlabel="")
        if d_df is not None:
            dflow_obs[key].plot(ax=ax, lw=line_weight, color="red", legend=False, xlabel="")
        ax.set_ylabel("Stage, m")
        t = flopy.plot.styles.heading(ax=ax, heading=f"Station: {key}")
        t.set_color("blue")
        if (idx, jdx) == (0, 0):
            flopy.plot.styles.graph_legend(ax=ax, loc="lower right", fontsize=6, title="none")
        
        if d_df is not None:
            df = d_df.copy()
        else:
            df = s_df.copy()
        if s_df is not None and d_df is not None:
            df["difference"] = dflow_obs[key]["D-FLOW FM"].values - sfincs_obs[key]["SFINCS"].values
        else:
            df["difference"] = np.nan
    
        idx += 1
        ax = axs[idx, jdx]
        df["difference"].plot(ax=ax, lw=line_weight, color="black", xlabel="")
        ax.set_ylabel("Stage\nDifference, m")

        idx += 1
        if idx > 1:
            idx = 0
            jdx += 1

    fig.savefig(ws / f"obs_ostations_summary{fig_ext}", dpi=300, transparent=transparent)

### Plot select observation locations (pstations)

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(
        ncols=3,
        nrows=4,
        layout="constrained",
        figsize=(11.5, 4),
        sharex=True,
        sharey=True
        )

    idx = 0
    jdx = 0

    for key in pstations:

        s_df = sfincs_obs[key]
        d_df = dflow_obs[key]
    
        ax = axs[idx, jdx]
        if s_df is not None:
            sfincs_obs[key].plot(ax=ax, lw=line_weight, color="blue", legend=False, xlabel="")
        if d_df is not None:
            dflow_obs[key].plot(ax=ax, lw=line_weight, color="red", legend=False, xlabel="")
        ax.set_ylabel("Stage, m")
        t = flopy.plot.styles.heading(ax=ax, heading=f"Station: {key}")
        t.set_color("green")
        if (idx, jdx) == (0, 0):
            flopy.plot.styles.graph_legend(ax=ax, loc="lower right", fontsize=6, title="none")
        
        if d_df is not None:
            df = d_df.copy()
        else:
            df = s_df.copy()
        if s_df is not None and d_df is not None:
            df["difference"] = dflow_obs[key]["D-FLOW FM"].values - sfincs_obs[key]["SFINCS"].values
        else:
            df["difference"] = np.nan
    
        idx += 1
        ax = axs[idx, jdx]
        df["difference"].plot(ax=ax, lw=line_weight, color="black", xlabel="")
        ax.set_ylabel("Stage\nDifference, m")

        idx += 1
        if idx > 3:
            idx = 0
            jdx += 1

    fig.savefig(ws / f"obs_pstations_summary{fig_ext}", dpi=300, transparent=transparent)